# Segunda atividade de programação

No código que se segue, você deverá completar as partes indicadas.

Note que o código está construído em OOP (Orientação ao Objeto), usando classes e métodos.

Leitura recomendada:

https://docs.microsoft.com/pt-br/learn/modules/python-object-oriented-programming/

In [1]:
from __future__ import annotations
from typing import List, Callable, Optional, TypeVar, Tuple
from functools import reduce
from random import random
from math import exp
from random import shuffle
import csv

In [2]:
# Funções auxiliares

def dot_product(xs: List[float], ys: List[float]) -> float:
    return sum(x * y for x, y in zip(xs, ys))

def sigmoid(x: float) -> float:
    return 1.0/(1.0 + exp(-x))

def derivative_sigmoid(x: float) -> float:
    sig: float = sigmoid(x)
    return sig * (1 - sig)

def normalize(dataset: List[List[float]]) -> None:
    for col_num in range(len(dataset[0])):
        column: List[float] = [row[col_num] for row in dataset]
        maximum: float = max(column)
        minimum: float = min(column)
        for row_num in range(len(dataset)):
            dataset[row_num][col_num] = (dataset[row_num][col_num] - minimum) / (maximum - minimum)

In [3]:
class Neuron:
    def __init__(self, weights: List[float], learning_rate: float,
    activation_function: Callable[[float], float],
    derivative_activation_function: Callable[[float], float]) -> None:
        self.weights: List[float] = weights
        self.activation_function: Callable[[float], float] = activation_function
        self.derivative_activation_function: Callable[[float], float] = \
            derivative_activation_function
        self.learning_rate: float = learning_rate
        self.output_cache: float = 0.0
        self.delta: float = 0.0

    def output(self, inputs: List[float]) -> float:
        self.output_cache = dot_product(inputs, self.weights)
        return self.activation_function(self.output_cache)

In [4]:
class Layer:
    def __init__(self, previous_layer: Optional[Layer], num_neurons: int,
    learning_rate: float, activation_function: Callable[[float], float],
    derivative_activation_function: Callable[[float], float]) -> None:
        self.previous_layer: Optional[Layer] = previous_layer
        self.neurons: List[Neuron] = []
        # following could all be one large list comprehension
        for i in range(num_neurons):
            if previous_layer is None:
                random_weights: List[float] = []
            else:
                random_weights = [random() for _ in range(len(previous_layer.neurons))]
            neuron: Neuron = Neuron(random_weights, learning_rate,
            activation_function, derivative_activation_function)
            self.neurons.append(neuron)
        self.output_cache: List[float] = [0.0 for _ in range(num_neurons)]

    def outputs(self, inputs: List[float]) -> List[float]:
        if self.previous_layer is None:
            self.output_cache = inputs
        else:
            self.output_cache = [n.output(inputs) for n in self.neurons]
        return self.output_cache

    # should only be called on output layer
    def calculate_deltas_for_output_layer(self, expected: List[float]) -> None:
        for n in range(len(self.neurons)):
            self.neurons[n].delta = self.neurons[n].derivative_activation_function(self.neurons[n].output_cache) \
                * (expected[n] - self.output_cache[n])

    # should not be called on output layer
    def calculate_deltas_for_hidden_layer(self, next_layer: Layer) -> None:
        for index, neuron in enumerate(self.neurons):
            next_weights: List[float] = [n.weights[index] for n in next_layer.neurons]
            next_deltas: List[float] = [n.delta for n in next_layer.neurons]
            sum_weights_and_deltas: float = dot_product(next_weights, next_deltas)
            neuron.delta = neuron.derivative_activation_function(neuron.output_cache) * sum_weights_and_deltas

Agora que temos uma implementação tanto do neurônio, quanto da camada, podemos passar para a construção da rede em si.

- A rede é apenas uma informação de estado: as camadas que ela administra.
- Abaixo temos uma classe `Network` que é responsável por inicializar as camadas que a compõem.
- O método `__init__()` aceita uma lista de `ints` que descreve a estrutura da rede.
> Exemplo: [2, 4, 2] descreve uma rede com 2 neurônios em sua camada de entrada, 4 na cama da oculta e 3 na de saída.
- Vamos assumir que todas as camadas irão usar a mesma função de ativação.

**Obs**: `T = TypeVar(T)` indica que qualquer tipo de dados é aceitável

In [5]:
T = TypeVar('T')

class Network:
    def __init__(self, layer_structure: List[int], learning_rate: float, 
                 activation_function: Callable[[float], float]=sigmoid,
                 derivative_activation_function: Callable[[float], float]=derivative_sigmoid) -> None:
        if len(layer_structure) < 3:
            raise ValueError("Error: Should be at least 3 layers: [1 input, 1 hidden, 1 output]")
        self.layers: List[Layer] = []
        # Construção da rede de entrada
        input_layer: Layer = Layer(None, layer_structure[0], learning_rate, activation_function, derivative_activation_function)
        self.layers.append(input_layer)
        
        # Construção das camadas oculta e de saída
        for previous, num_neurons in enumerate(layer_structure[1::]):
            next_layer = Layer(self.layers[previous], num_neurons, learning_rate, activation_function, derivative_activation_function)
            self.layers.append(next_layer)
    
    def outputs(self, input: List[float]) -> List[float]:
        """ Fornece dados de entrada para a primeira camada; em seguida, a saída da primeira
        é forneceida como entrada para a segunda, a saída da segunda para a terceira, e assim
        segue de maneira sucessiva.
        """
        return reduce(lambda inputs, layer: layer.outputs(inputs), self.layers, input)
    
    def backpropagate(self, expected: List[float]) -> None:
        """Função responsável por calcular os deltas para todos os neurônios da rede.
        Calcula as mudanças em cada neurônio com base nos erros da saída, em comparação
        com a saída esperada.
        """
        # delta para cada neurônio da camada de saída
        last_layer: int = len(self.layers) - 1
        self.layers[last_layer].calculate_delas_for_out_put_layer(expected)
        
        # delta para as camadas ocultas, na ordem inversa
        for l in range(last_layer - 1, 0, -1):
            self.layers[l].calculate_deltas_for_hidden_layers(self.layers[l+1])
    
    def update_weights(self) -> None:
        """Atualiza os pesos da rede usando os resultados produzidos pelo
        método backpropagate.
        """
        for layer in self.layers[1:]: # pulamos a camada de entrada
            for neuron in layer.neurons:
                for w in range(len(neuron.weights)):
                    neuron.weights[w] = neuron.weights[w] + (neuron.learning_rate * (layer.previous_layer.output_cache[w]) * neuron.delta)
    
    # Já temos toda a estrutura montada. O que precisamos agora são das funções para treino e validação.
    def train(self, inputs: List[[float]], expecteds: List[[float]]) -> None:
        """Usa o resultado de outputs(), obtidos a partir de várias entradas e
        comparados com expecteds, para fornecer ao backpropagete() e update_weights()
        """
        for location, xs in enumerate(inputs):
            ys: List[float] = expecteds[location]
            outs: List[float] = self.outputs(xs)
            self.backpropagate(ys)
            self.update_weights()
            
    def validate(self, inputs: List[[float]], expecteds: List[T],
                 interpret_output: Callable[[List[float]], T]) -> Tuple[int, int, float]:
        """Para resultados genéricos que exijam classificação, esta função devolve o número
        de tentativascorretas e a percentagem delas em relação ao total.
        """
        correct: int = 0
        for input, expected in zip(inputs, expecteds):
            result: T = interpret_output(self.outputs(input))
            if result == expected: correct += 1
            percentage: float = correct / len(inputs)
            return correct, len(inputs), percentage

# Teste da implementação

Vamos usar um dataset bem popular chamado Iris, que contem 3 diferentes classes de uma mesma flor: Iris-setosa, Iris-versicolor e  Iris-virginica

In [6]:
if __name__ == "__main__":
    iris_parameters: List[List[float]] = []
    iris_classifications: List[List[float]] = []
    iris_species: List[str] = []
    with open('iris.csv', mode='r') as iris_file:
        irises: List = list(csv.reader(iris_file))
        shuffle(irises) # get our lines of data in random order
        for iris in irises:
            parameters: List[float] = [float(n) for n in iris[0:4]]
            iris_parameters.append(parameters)
            species: str = iris[4]
            if species == "Iris-setosa":
                iris_classifications.append([1.0, 0.0, 0.0])
            elif species == "Iris-versicolor":
                iris_classifications.append([0.0, 1.0, 0.0])
            else:
                iris_classifications.append([0.0, 0.0, 1.0])
            iris_species.append(species)
    normalize(iris_parameters)

    iris_network: Network = Network([4, 6, 3], 0.3)

    def iris_interpret_output(output: List[float]) -> str:
        if max(output) == output[0]:
            return "Iris-setosa"
        elif max(output) == output[1]:
            return "Iris-versicolor"
        else:
            return "Iris-virginica"

    # train over the first 140 irises in the data set 50 times
    iris_trainers: List[List[float]] = iris_parameters[0:140]
    iris_trainers_corrects: List[List[float]] = iris_classifications[0:140]
    for _ in range(50):
        iris_network.train(iris_trainers, iris_trainers_corrects)

    # test over the last 10 of the irises in the data set
    iris_testers: List[List[float]] = iris_parameters[140:150]
    iris_testers_corrects: List[str] = iris_species[140:150]
    iris_results = iris_network.validate(iris_testers, iris_testers_corrects, iris_interpret_output)
    print(f"{iris_results[0]} correct of {iris_results[1]} = {iris_results[2] * 100}%")

AttributeError: 'Layer' object has no attribute 'calculate_delas_for_out_put_layer'